<a href="https://colab.research.google.com/github/hank199599/deep_learning_keras_log/blob/main/Chapter8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 使用LSTM產生文字資料

## 生成序列資料
訓練神經網路為先前輸入的序列資料產生接續的資料，  
該神經網路被稱為語言模型，他會去捕捉語言的淺在空間(latent space)。  
* 條件資料(conditioning data)：初始的文字字串

## 字元等級的語言模型 (charactor-level neural language model)
使用LSTM層，從文字資料庫(text corpus)中萃取N個字元的字串作為輸入，  
並訓練模型預設第N+1個字元。  
  
**模型的輸出**：所有可能的字元經softmax運算後的結果  
![pic 8-1](https://raw.githubusercontent.com/hank199599/deep_learning_keras_log/main/pictures/8-1.png)

## 取樣策略的重要性
1. **greedy sampling** (貪婪取樣)：在所有可能字元中選擇最可能的下一個字元，但易導致重複的預測字串
2. **stochastic sampling** (隨機取樣): 自下一個字元的機率分布中取樣，並導入隨機性

### 控制隨機性 control the amount of randomness
在極端情形下，若分布不具隨機性會使抽樣結果則使該方法無效化
* 分布機率具最大熵(entorpy)：每個字元具有相同的可能性並具有最大的隨機性
* 分布機率具最小熵(entorpy)：字元分布幾乎不具備隨機性

#### softmax temperature：取樣機率分布熵的指標
代表下一個字元的不確定性

##### 範例
三個可能會出現的字元 a,b,c  
a 出現機率為 80%  
b,c 出現機率為 20%   
  
建立一個function針對不同的temperature設定，對機率分布重新加權並計算出新的機率分布

In [ ]:
import numpy as np

def reweight_distribution(original_distribution,temperature=0.5):
  distribution = np.log(original_distribution)/temperature
  distribution = np.exp(distribution)
  return distribution / np.sum(distribution) #傳回重新加權
  
  # 因為重新加權後，機率分布之總和可能不再是1。因此再將其除以總合以滿足總合為1

In [ ]:
ori_dstri = np.array([0.8,0.1,0.1])

In [ ]:
new_dstri = reweight_distribution(ori_dstri,temperature=0.01)
print(new_dstri)

[1.00000000e+00 4.90909347e-91 4.90909347e-91]


In [ ]:
new_dstri = reweight_distribution(ori_dstri,temperature=2)
print(new_dstri)

[0.58578644 0.20710678 0.20710678]


In [ ]:
new_dstri = reweight_distribution(ori_dstri,temperature=10)
print(new_dstri)

[0.38102426 0.30948787 0.30948787]


![pic 8-2](https://raw.githubusercontent.com/hank199599/deep_learning_keras_log/main/pictures/8-2.png)  
使用較高temperature使亂度上升，產生更高的隨機性  
相反地，越低的溫度會使廳選出的樣本偏向特定一方

## 實現字元級LSTM文字資料生成 (本機端上操作)

In [ ]:
import keras
import numpy as np

path = keras.utils.get_file(
    'nietzsche.txt',
    origin = 'https://s3.amazonaws.com/text-datasets/nietzsche.txt'
)
text = open(path).read().lower()
print('Corpus length:',len(text))

606208/600901 [==============================] - 0s 1us/step
Corpus length: 600893


以每3個字元作為間隔，萃取出一段段長度為maxlen的部分重疊序列資料

### 向量化字元序列

In [ ]:
maxlen = 60
step = 3
sentences = []
next_chars = []

for i in range(0,len(text)-maxlen,step):
  sentences.append(text[i:i+maxlen])
  next_chars.append(text[i+maxlen])
print('Number of sequences:',len(sentences))

chars = sorted(list(set(text)))
print('Unique charactors:',len(chars))

# 將各個字元對應到"chars"串列中的索引值，成為字典格式
char_indices = dict((char,char.index(char)) for char in chars)

# 將字元經One-Hot編碼為二元陣列
print('Vectorization....')
x = np.zeros((len(sentences),maxlen,len(chars)),dtype=np.bool)
y = np.zeros((len(sentences),len(chars)),dtype=np.bool)

for i,sentence in enumerate(sentences):
  for t , char in enumerate(chars):
    x[i,t,char_indices[char]] = 1
  y[i,char_indices[next_chars[i]]] = 1

print(x.shape)
print(y.shape)


Number of sequences: 200278
Unique charactors: 57
Vectorization....
(200278, 60, 57)
(200278, 57)


## 建立神經網路

In [ ]:
from keras import layers 

model = keras.models.Sequential()
model.add(layers.LSTM(128,input_shape=(maxlen,len(chars))))
model.add(layers.Dense(len(chars),activation='softmax')) # 預測字元種類，共57種

#### 模型編譯設定

In [ ]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy',optimizer=optimizer)

### 訓練語言模型並從中取樣
1. 給定到目前為止生成的文字，從模型中繼下一個字元的機率分布
2. 將分布重新調整到一特定的temperature
3. 根據重新加權的分布隨機取樣取得下一個字元
4. 在現有的文字字尾加入新字元

In [ ]:
def sample(preds,temperature=1.0):
  preds = np.asarray(preds).astype('float64')
  # ======== 重新加權計算熵 ===============
  preds = np.log(preds) / temperature
  exp_preds = np.exp(preds)
  preds = exp_preds / np.sum(exp_preds)
  
  probas = np.random.multinomial(1,preds,1)
  return np.argmax(probas) # 取出多項式分佈的結果

#### 循環文字的生成

In [ ]:
import random 
import sys

for epoch in range(1,60):
  print('epoch',epoch)
  model.fit(x,y,batch_size=128,epochs=1)

  # 隨機選擇文本中的某段60個字元的文字 (初始文字)
  start_index = random.randint(0,len(text)-maxlen-1)
  generated_text = text[start_index:start_index+maxlen]
  
  print('-----random initial text:"',generated_text,'"')


  for temperature in [0.2,0.5,1.0,1.2]:
    print('-----temperature:',temperature)
    sys.stdout.write(generated_text)

    # 每個temperature生成400個字元
    for i in range(400):
      sampled = np.zeros((1,maxlen,len(chars)))
      for t,char in enumerate(generated_text):
        sampled[0,t,char_indices[char]] = 1.
      
      preds = model.predict(sampled,verbose=0)[0]
      next_index = sample(preds,temperature)
      next_char = chars[next_index]
      generated_text += next_char
      generated_text = generated_text[1:]
      sys.stdout.write(next_char)



# DeepDream
![10 iterations of applying DeepDream](https://upload.wikimedia.org/wikipedia/commons/8/89/Aurelia-aurita-3-0009.jpg)
一種藝術圖片修改技術，使用**反向卷積神經網路學習**

### 載入預先訓練的[Inception V3](https://arxiv.org/pdf/1512.00567v3.pdf) 模型


In [16]:
from tensorflow.keras.applications import inception_v3
from tensorflow.keras import backend as K
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

K.set_learning_phase(0) # 終止所有訓練相關的操作
model = inception_v3.InceptionV3(weights='imagenet',include_top=False)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/backend.py:434: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '


## 計算損失
選擇一組高層的啟動函式，取其輸出的L2 Norm。  
並以它們的加權總合做為損失函數  
藉由**梯度上升法**尋求最大化的損失

建立字典並將層名稱對應到其損失函數的**貢獻係數**(value)

In [17]:
layer_contributions={
  'mixed2':0.2,
  'mixed3':2.,  
  'mixed4':2.,
  'mixed5':1.5,
}

### 定義損失最大化

In [18]:
layer_dict = dict([(layer.name,layer) for layer in model.layers])

loss = K.variable(0.)
for layer_name in layer_contributions:
  coeff = layer_contributions[layer_name]
  activation = layer_dict[layer_name].output
  scaling = K.prod(K.cast(K.shape(activation),'float32'))

  loss = loss + coeff * K.sum(K.square(activation[:,2:-2,2:-2,:])) / scaling

### 梯度上升處理程序

In [19]:
dream = model.input
print(dream.shape)

(None, None, None, 3)


In [21]:

grads = K.gradients(loss,dream)[0]
grads /= K.maximum(K.mean(K.abs(grads)),1e-7) # 正規化梯度

# 在給定輸入圖片的情況下，自定一個Keras已取得損失值與梯度值
outputs = [loss,grads]
fetch_loss_and_grads = K.function([dream],outputs)

def eval_loss_and_grads(x):
  outs = fetch_loss_and_grads([x])
  loss_value = outs[0]
  grad_value = outs[1]
  return loss_value,grad_value

def gradient_ascent(x,iterations,step,max_loss=None):
  for i in range(iterations):
    if loss_value is not None and loss_value > max_loss:
      break
    print('...Loss value ar',i,':',loss_value)
    print('...grad value ar',i,':',grad_values)
    x += step*grad_values
  return

### DeepDream 演算法
1. 定義一個處理圖片的比例(Scale)串列
2. 每個連續的比例比前一個比例大1.4倍，在每次梯度上升後將圖片放大40%在繼續進行梯度上升
為避免每次連續放大後損失大量圖片細節，將損失的細節重新融入圖片  
![pic 8-3](https://raw.githubusercontent.com/hank199599/deep_learning_keras_log/main/pictures/8-3.png) 

In [22]:
import scipy
from keras.preprocessing import image 

# 對葡片進行預處理
def preprocess_image(image_path):
  img = image.load_img(image_path)
  img = img.img_to_array(img)
  print(img.shape)
  img = np.expand_dims(img,axis=0)
  print(img.shape)
  img = inception_v3.preprocess_input(img)
  return img

# 將Inception V3所做的預處理進行反向操作，轉回圖片格式
def deprocess_image(x):
  if K.image_data_format()=='channels_first':
    x = x.reshape((3,x.shape[2],x.shape[3]))
    x = x.transpose((1,2,0))
  else:
    x = x.reshape((x,shape[1],x.shape[2],3))
  x /= 2.
  x *= 0.5
  x *= 255.
  x = np.clip(x,0,255).astype('unit8') # 將數字限制在0-255之間
  return x

# 進行圖片比例的縮放
def resize_img(img,size):
  img = np.copy(img)
  factors = (1,
        float(size[0])/img.shape[1],
        float(size[1])/img.shape[2],
        1
        )
  return scipy.ndimage.zoom(img,factors,order=1) #以樣條插值法的技術對圖片進行縮放


# 儲存圖片，於儲存前反轉Inception V3所做的預處理
def save_img(img,fname):
  pil_img = deprocess_image(np.copy(img))
  scipy.misc.imsave(fname,pil_img)

### 在不同的連續比例的圖片上執行梯度上升

In [23]:
step = 0.01
num_octave = 3
octave_scale = 1.4
iterations = 20
max_loss = 10.

base_image_path = 'original_photo_deep_dream.jpg'
img = preprocess_image(base_image_path) # 載入圖片並進行預處理
original_shape = img.shape[1:3]
successive_shapes = [original_shape]

for i in range(1,num_octave):
  shape = tuple([int(dim / (octave_scale ** i)) for dim in original_shape])
  successive_shapes.append(shape)
successive_shapes = successive_shapes[::-1] # 反轉寬高比例list，使他們按照大小順序遞增

original_img = np.copy(img)
shrunk_original_img = resize_img(img,successive_shapes[0])

for shape in successive_shapes: # 開始逐次放大圖片
  print('Preprocessing image shape',shape)
  img = resize_img(img,shape)
  img = gradient_ascent(img,iterations=iterations,step=step,max_loss=max_loss) # 執行梯度上升
  upscaled_shrunk_original_img = resize_img(shrunk_original_img,shape)

  same_size_original = resize_img(original_img,shape) # 將原始圖片縮小至目前比例(大→小)
  lost_detail = same_size_original - upscaled_shrunk_original_img # 相減求得損失的細節
  img += lost_detail # 將損失的細節放回圖片中
  shrunk_original_img = resize_img(original_img,shape)
  save_img(img,frame='dream_at_scale_'+str(shape)+'.png')

save_img(img,fname='final_dream.png')

FileNotFoundError: ignored

# 神經風格轉換

# 使用變分自編碼器 Variational Autoencoders 生成圖像

# 生成對抗神經網路簡介 Generative Adversarial Network
